In [ ]:
%load_ext autoreload
%autoreload 2

In [72]:
import pandas as pd
from autobook.libgen import Libgen

data = Libgen(q="cloud cuckoo land")
df = data.search()

In [73]:
df.filter("Anthony Doerr")

NameError: name 'df' is not defined

In [ ]:
df.to_json()